In [3]:
import os
from PIL import Image
from pathlib import Path
from dotenv import load_dotenv

os.chdir('/Users/glenn_hyh/Documents/github/bc-wildfire-prediction')
from mine_seg_sat import dataset
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

load_dotenv()

# data_path = Path(os.getenv("prepare_dataset_folder"))
# processed_path = Path(os.getenv("processed_images_folder"))
data_path = Path('prepared_dataset')
processed_path = Path('processed_images_folder')

split = "test"  # Adjust as needed for 'val' or 'test'

# Ensure the processed data folder exists
processed_path.mkdir(parents=True, exist_ok=True)

# Instantiate the dataset
forest_sat_dataset = dataset.MineSATDataset(split=split, data_path=data_path)

# Iterate over all dataset indices
for index in range(len(forest_sat_dataset)):
    # Get the filepaths for the current index
    filepath = forest_sat_dataset.filepaths[index]

    # Extract the original directory name
    original_dir_name = Path(filepath).name

    # Get the transformed images
    images_dict = forest_sat_dataset.get_images(
        index)  # Ensure you call the correct method

    # Create a new directory path for the processed images
    processed_dir_path = processed_path / original_dir_name
    processed_dir_path.mkdir(parents=True, exist_ok=True)

    for img_type, image in images_dict.items():
        # Skip saving the mask if you only want the indices and RGB images
        if img_type == "Mask":
            continue

        if img_type == "NBR":
            # Apply color mapping for NBR
            plt.imshow(image, cmap=plt.cm.RdYlGn, vmin=-1, vmax=1)
            plt.colorbar(orientation='vertical')
            plt.axis('off')

            # Save the current figure to a numpy array
            fig = plt.gcf()
            plt.draw()
            image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            image_np = image_np.reshape(
                fig.canvas.get_width_height()[::-1] + (3,))

            # Convert numpy array to PIL Image
            pil_image = Image.fromarray(image_np)

            # Define the filename, replacing .tif with .png for NBR
            filename = f"{img_type}.png"

            # Close the figure to free memory
            plt.close(fig)
        else:
            # Normalize the image data to 0-255 for other image types
            # Clip to the range you want
            image = np.clip(image, 0, np.max(image))
            image_8bit = ((image - np.min(image)) /
                          (np.max(image) - np.min(image)) * 255).astype('uint8')

            # If the image has more than one channel, convert it to RGB
            if image_8bit.ndim > 2 and image_8bit.shape[2] > 3:
                # Convert multi-band images (e.g., 4 bands) to RGB (3 bands) before saving as JPEG
                image_8bit = image_8bit[:, :, :3]

            # Create the PIL Image from the numpy array
            pil_image = Image.fromarray(image_8bit)

            # Define the filename, replacing .tif with .jpg for other image types
            filename = f"{img_type}.jpg"

        # Define the full path for the file
        filepath = processed_dir_path / filename

        # Save the image
        pil_image.save(filepath)


{'NBR': array([[ 0.03105304, -0.05873303,  0.00112699, ...,  0.02024897,
         0.03901227, -0.01485664],
       [-0.0665626 , -0.01531516,  0.05096825, ...,  0.07238229,
         0.02853571, -0.06570817],
       [-0.08133649,  0.02399861, -0.0732491 , ...,  0.03032296,
         0.0356737 , -0.08698426],
       ...,
       [-0.01927352,  0.0454592 ,  0.04190185, ...,  0.23486258,
         0.09408951, -0.12420527],
       [-0.03833886, -0.0786758 ,  0.01379896, ...,  0.10029278,
        -0.18115848, -0.33105898],
       [-0.05142457, -0.11722238,  0.02069819, ..., -0.43993668,
        -0.22435925, -0.14573593]]), 'NDVI': array([[ 0.38258743,  0.36918695,  0.3956431 , ...,  0.644872  ,
         0.63545172,  0.63927068],
       [ 0.44013412,  0.41318856,  0.44046243, ...,  0.63328758,
         0.68502372,  0.70625385],
       [ 0.33990525,  0.42421274,  0.53972438, ...,  0.66278748,
         0.64357443,  0.71159003],
       ...,
       [ 0.28104575,  0.19741665,  0.15361225, ...,  0.219

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_25157/588929264.py:59: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/mine_seg_sat/dataset.py:251: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)
/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_25157/588929264.py:75: RuntimeWarning: invalid value encountered in divide
  image_8bit = ((image - np.min(image)) /
/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_25157/588929264.py:76: RuntimeWarning: invalid value encountered in cast
  (np.max(image) - np.min(image)) * 255).astype('uint8')
